In [1]:
import urllib
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [5]:
# read in novel data scraped from Kimono labs 
novels = pd.read_csv("../rawdata/novel_names_urls_dates.csv")
novels.head()

,Novel Name.href,Novel Name.text,Novel Date,index,url
0,http://nanowrimo.org/participants/nicaless/nov...,Lauren's Birthday,November 2015,1,http://nanowrimo.org/participants/nicaless/novels
1,http://nanowrimo.org/participants/nicaless/nov...,A Mystery in the Kingdom of Aermon,November 2014,2,http://nanowrimo.org/participants/nicaless/novels
2,http://nanowrimo.org/participants/rachel-b-moo...,Finding Fortunato,November 2015,3,http://nanowrimo.org/participants/rachel-b-moo...
3,http://nanowrimo.org/participants/rachel-b-moo...,The Residency,November 2014,4,http://nanowrimo.org/participants/rachel-b-moo...
4,http://nanowrimo.org/participants/rachel-b-moo...,The Jew From Fortunato,November 2013,5,http://nanowrimo.org/participants/rachel-b-moo...


In [6]:
novels = novels[['Novel Name.text', 'Novel Date', 'Novel Name.href', 'url']]
novels = novels.rename(columns={'Novel Name.text': 'Novel Name', 'Novel Date': 'Novel Date', 'Novel Name.href': 'Novel Url', 'url':'Writer Url'})
novels.head()

,Novel Name,Novel Date,Novel Url,Writer Url
0,Lauren's Birthday,November 2015,http://nanowrimo.org/participants/nicaless/nov...,http://nanowrimo.org/participants/nicaless/novels
1,A Mystery in the Kingdom of Aermon,November 2014,http://nanowrimo.org/participants/nicaless/nov...,http://nanowrimo.org/participants/nicaless/novels
2,Finding Fortunato,November 2015,http://nanowrimo.org/participants/rachel-b-moo...,http://nanowrimo.org/participants/rachel-b-moo...
3,The Residency,November 2014,http://nanowrimo.org/participants/rachel-b-moo...,http://nanowrimo.org/participants/rachel-b-moo...
4,The Jew From Fortunato,November 2013,http://nanowrimo.org/participants/rachel-b-moo...,http://nanowrimo.org/participants/rachel-b-moo...


In [7]:
# get urls for novel pages
novel_urls = pd.read_csv("../rawdata/novel_urls.csv", header = None)
novel_urls = novel_urls.rename(columns={0: 'Urls'})

novel_urls.head()

,Urls
0,http://nanowrimo.org/participants/nicaless/nov...
1,http://nanowrimo.org/participants/nicaless/nov...
2,http://nanowrimo.org/participants/rachel-b-moo...
3,http://nanowrimo.org/participants/rachel-b-moo...
4,http://nanowrimo.org/participants/rachel-b-moo...


In [6]:
num_submissions = []
avg_submissions = []
min_submissions = []
min_day = []
max_submissions = []
max_day = []
std_submissions = []

num_consec_sub = []

testnovels = ['http://nanowrimo.org/participants/nicaless/novels/lauren-s-birthday/stats', 'http://nanowrimo.org/participants/nicaless/novels/a-mystery-in-the-kingdom-of-aermon/stats']

for u in novel_urls['Urls']:
#for u in testnovels:
    print u
    page =  urllib.urlopen(u).readlines()
    for line in page:
        # find the line that starts with rawCamperData. rawCamperData is the javascript variable that contains the daily word count data for the novel
        if line[10:23] == "rawCamperData":
            # split the string to get the wc submission values
            mystr =  line[27:len(line)-3]
            mystr = mystr.split(",")
            mystr = list(mystr)
            mystr = map(int, mystr)
            
            # get number of submissions and actual (non-cumulative) wc submission values
            submissions = {}
            # use these vars to count the number of consecutive submissions (at least 2 non-zero word count submissions in a row)
            consec_total = 0
            consec = 0
            
            # walk through the array of submission values
            for i in range(len(mystr)):
                if i == 0: # if it's the first submission value
                    if mystr[i] > 0: # if it's a non-zero submission start counting consecutive submissions
                        consec = consec + 1
                    submissions[i+1] = mystr[i] 
                elif mystr[i] == mystr[i-1]: # if the word count equals yesterday's word count, then it is a zero word count submission
                    if consec > 1:
                        consec_total = consec_total + consec # if at least 2 non-zero word count submissions in a row, add it to count of total consecutive submission
                    consec = 0 # reset counter
                else: 
                    consec = consec + 1 # if word count is different from yesterday up the counter of consecutive submissions
                    new_sub = mystr[i] - mystr[i-1] # get the difference in word count to find the actual submission
                    submissions[i+1] = new_sub # append the submission
            if consec > 1:
                consec_total = consec_total + consec #after loop is finished, check again if final submission was part of a series of consecutive submissions and add it to total
            
            # append all the calculated values of this novel to the arrays
            num_submissions.append(len(submissions))
            avg_submissions.append(np.mean(submissions.values()))
            min_submissions.append(min(submissions.values()))
            min_day.append(min(submissions, key=submissions.get))
            max_submissions.append(max(submissions.values()))
            max_day.append(max(submissions, key=submissions.get))
            std_submissions.append(np.std(submissions.values()))
            num_consec_sub.append(consec_total)

http://nanowrimo.org/participants/nicaless/novels/lauren-s-birthday/stats
http://nanowrimo.org/participants/nicaless/novels/a-mystery-in-the-kingdom-of-aermon/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/finding-fortunato/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/the-residency-579624/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/the-jew-from-fortunato/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/perdido/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/skipping-rocks-at-naskeag/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/untitled-well-i-am-sure-it-had-a-title-at-the-time-but-i-can-t-remember-it/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/ventajas-ventanas/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/untitled-andre-levinsky-backstory/stats
http://nanowrimo.org/participants/rachel-b-moore/novels/the-brothers-bolano/stats
http://nanowrimo.org/participants/

In [7]:
#append the arrays to the novel data
novels["Num Submissions"] = num_submissions
novels["Avg Submission"] = avg_submissions
novels["Min Submission"] = min_submissions
novels["Min Day"] = min_day
novels["Max Submission"] = max_submissions
novels["Max Day"] = max_day
novels["Standard Deviation"] = std_submissions
novels['Consecutive Submissions'] = num_consec_sub
novels.head()

,Novel Name,Novel Date,Novel Url,Writer Url,Num Submissions,Avg Submission,Min Submission,Min Day,Max Submission,Max Day,Standard Deviation,Consecutive Submissions
0,Lauren's Birthday,November 2015,http://nanowrimo.org/participants/nicaless/nov...,http://nanowrimo.org/participants/nicaless/novels,12,2019.083333,296,4,10699,29,2787.599010,10
1,A Mystery in the Kingdom of Aermon,November 2014,http://nanowrimo.org/participants/nicaless/nov...,http://nanowrimo.org/participants/nicaless/novels,14,3637.071429,299,2,24935,28,6235.712933,12
2,Finding Fortunato,November 2015,http://nanowrimo.org/participants/rachel-b-moo...,http://nanowrimo.org/participants/rachel-b-moo...,28,1807.250000,148,27,5885,1,1515.170986,28
3,The Residency,November 2014,http://nanowrimo.org/participants/rachel-b-moo...,http://nanowrimo.org/participants/rachel-b-moo...,21,2401.190476,83,29,6265,22,1777.616720,20
4,The Jew From Fortunato,November 2013,http://nanowrimo.org/participants/rachel-b-moo...,http://nanowrimo.org/participants/rachel-b-moo...,20,2072.350000,68,3,8079,23,2198.931701,17


In [8]:
novels.to_csv("../rawdata/novels_wc_info.csv")